In [2]:
#! pip install jupyterlab-widgets==1.1.1 ipywidgets==7.7.1
#! pip install plotly==5.22.0

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from sklearn.preprocessing import StandardScaler
import umap
import os

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


### In this example, the embeddings are generated from a Barlow Twin model trained on the Left CINGULATE region on UKB for the UKB and HCP datasets.

 -For the HCP dataset, the associated buckets can be found in: 

/neurospin/dico/data/deep_folding/current/datasets/hcp/crops/2mm/CINGULATE/mask/Lbuckets

 -For the UKB dataset, the buckets are not generated yet, but it would be found in:

/neurospin/dico/data/deep_folding/current/datasets/UkBioBank/crops/2mm/CINGULATE/mask/Lbuckets

### Load your data

In [113]:
#embeddings_HCP = pd.read_csv("/neurospin/dico/adufournet/Runs/02_Heritability_Left_PCS_HCP/Output/2024-05-13/09-33-29_206/hcp_epoch60_embeddings/full_embeddings.csv", index_col=0)
#embeddings_UKB = pd.read_csv("/neurospin/dico/adufournet/Runs/02_Heritability_Left_PCS_HCP/Output/2024-05-13/09-33-29_206/UKB_epoch60_embeddings/full_embeddings.csv", index_col=0)
embeddings_HCP = pd.read_csv("/neurospin/dico/adufournet/Runs/01_Heritability_Right_PCS_HCP/Output/2024-05-28/10-37-46_0/HCP_random_epoch60_embeddings/full_embeddings.csv", index_col=0)
embeddings_UKB = pd.read_csv("/neurospin/dico/adufournet/Runs/01_Heritability_Right_PCS_HCP/Output/2024-05-28/10-37-46_0/UKB_random_epoch60_embeddings/full_embeddings.csv", index_col=0)
embeddings_UKB.head()

embeddings_HCP = embeddings_HCP
embeddings_UKB = embeddings_UKB
embeddings_UKB.head()

,dim1,dim2,dim3,dim4,dim5,dim6,dim7,dim8,dim9,dim10,...,dim247,dim248,dim249,dim250,dim251,dim252,dim253,dim254,dim255,dim256
ID,,,,,,,,,,,,,,,,,,,,,
sub-1000021,-26.721450,-9.475343,35.959435,-2.331884,16.956598,-4.985778,-33.438705,14.604317,-2.730797,27.709997,...,-3.539915,-4.459958,-14.570611,1.635946,6.412560,6.935651,-6.811207,18.208925,13.374589,-7.218053
sub-1000458,-12.061327,-8.985274,49.456640,2.321947,25.568815,-3.107683,-3.816329,4.817796,-1.004531,7.084780,...,15.538760,-20.475363,-29.890331,-1.455647,-10.014833,21.725365,-1.845990,6.680744,2.144575,-10.844675
sub-1000575,-4.316093,-23.662600,44.971565,-9.912062,1.373107,7.666853,26.839014,4.749241,-15.865440,-0.028321,...,27.742048,19.149477,-3.228610,-8.354598,10.213907,-3.500815,0.049003,7.247147,-5.031011,-2.936263
sub-1000606,-37.049550,-1.207152,74.084785,-3.903676,-7.867699,-0.640225,-3.337830,-13.962615,-8.070219,-7.474173,...,31.670496,-11.351337,-8.907484,0.153167,-1.749980,-2.951459,-0.911485,-26.038542,-24.271719,-8.886850
sub-1000963,-14.658484,-5.484344,55.349876,5.966642,-10.079696,-3.637263,-8.565697,19.298077,-16.087200,31.489850,...,25.921968,-7.085027,-26.555103,2.508141,3.510635,22.999752,-2.648190,12.479370,2.825703,-9.746050


### Scale your data

In [114]:
scaler = StandardScaler()
scaler.fit(embeddings_UKB)

scl_bdd_hcp = scaler.transform(embeddings_HCP)
scl_bdd_ukb = scaler.transform(embeddings_UKB)

### 2D UMAP

In [115]:
reducer2D = umap.UMAP(n_components=2)

reducer2D.fit(scl_bdd_ukb)

bdd_2D_HCP = reducer2D.transform(scl_bdd_hcp)
bdd_2D_UKB = reducer2D.transform(scl_bdd_ukb)

In [135]:
bdd_2D_HCP = pd.DataFrame(bdd_2D_HCP, columns=['Dim 1', 'Dim 2'])
bdd_2D_UKB = pd.DataFrame(bdd_2D_UKB, columns=['Dim 1', 'Dim 2'])

bdd_2D_HCP['Dataset'] = 'hcp'
bdd_2D_UKB['Dataset'] = 'UkBioBank'

bdd_2D_HCP['ID'] = embeddings_HCP.index
bdd_2D_UKB['ID'] = embeddings_UKB.index

bdd_2D_All = pd.concat([bdd_2D_UKB,bdd_2D_HCP], axis=0)

# Calculate the centroid
centroid_dim1 = bdd_2D_All['Dim 1'].mean()
centroid_dim2 = bdd_2D_All['Dim 2'].mean()
centroid_row = {
    'Dim 1': centroid_dim1,
    'Dim 2': centroid_dim2,
    'Dataset': 'centroid',
    'ID': 'centroid'
}

# Append the centroid row to the dataframe
bdd_2D_All = bdd_2D_All.append(centroid_row, ignore_index=True)

# Calculate the distance to the centroid for each point
bdd_2D_All['Euclidean_Dist_to_Centroid'] = np.sqrt((bdd_2D_All['Dim 1'] - centroid_dim1)**2 + (bdd_2D_All['Dim 2'] - centroid_dim2)**2)
bdd_2D_All['Manhattan_Dist_to_Centroid'] = abs(bdd_2D_All['Dim 1'] - centroid_dim1)+ abs(bdd_2D_All['Dim 2'] - centroid_dim2)


bdd_2D_All.head()

,Dim 1,Dim 2,Dataset,ID,Euclidean_Dist_to_Centroid,Manhattan_Dist_to_Centroid
0,7.380236,3.499890,UkBioBank,sub-1000021,4.350174,5.063930
1,2.269651,4.975546,UkBioBank,sub-1000458,1.081072,1.522313
2,3.077432,7.618707,UkBioBank,sub-1000575,3.333698,3.357693
3,5.466866,5.483536,UkBioBank,sub-1000606,2.651632,3.563792
4,1.099855,4.354158,UkBioBank,sub-1000963,2.002849,2.070720


In [139]:
#bdd_2D_All["Euclidean_Dist_to_Centroid"].hist(bins=60, alpha=0.6, label='Euclidean')
#bdd_2D_All["Manhattan_Dist_to_Centroid"].hist(bins=60, alpha=0.6, label='Manhattan')
#plt.legend()
#plt.show()

### Plot it in the notebook or write a html file

In [125]:
subject_id_list = []
dataset_name_list = []

In [126]:
# Create the scatter plot using plotly express
fig = px.scatter(
    bdd_2D_All, x='Dim 1', y='Dim 2', 
    color='Dataset',
    title='2D UMAP HCP and UKB',
    labels={'0': 'dim 1', '1': 'dim 2'},
    hover_data= ['Dataset', 'ID'],
    opacity=0.5,
    width=800, height=600
)

# Convert the figure to a FigureWidget
f = go.FigureWidget(fig)

# Define the callback function
def click_callback(trace, points, selector):
    for trace_index in range(len(f.data)):
        if trace_index == points.trace_index:
            customdata = f.data[trace_index].customdata
            for i in points.point_inds:
                point_dataset, point_id = customdata[i]
                print(f"Clicked point ID: {point_id}, Dataset: {point_dataset}")
                subject_id_list.append(point_id)
                dataset_name_list.append(point_dataset)

# Attach the callback to the on_click event for all traces
for trace in f.data:
    trace.on_click(click_callback)

# Display the figure widget
f

FigureWidget({
    'data': [{'customdata': array([['UkBioBank', 'sub-1000021'],
                                   ['UkBioBank', 'sub-1000458'],
                                   ['UkBioBank', 'sub-1000575'],
                                   ...,
                                   ['UkBioBank', 'sub-6023847'],
                                   ['UkBioBank', 'sub-6024038'],
                                   ['UkBioBank', 'sub-6024754']], dtype=object),
              'hovertemplate': ('Dataset=%{customdata[0]}<br>Di' ... '{customdata[1]}<extra></extra>'),
              'legendgroup': 'UkBioBank',
              'marker': {'color': '#636efa', 'opacity': 0.5, 'symbol': 'circle'},
              'mode': 'markers',
              'name': 'UkBioBank',
              'showlegend': True,
              'type': 'scattergl',
              'uid': 'f0461c9c-6c2c-4e59-b9b1-bc71d111cd98',
              'x': array([7.380236 , 2.2696507, 3.0774317, ..., 5.0545206, 2.2635067, 1.8328245],
               

Clicked point ID: sub-1160330, Dataset: UkBioBank
Clicked point ID: sub-2938526, Dataset: UkBioBank
Clicked point ID: sub-4595658, Dataset: UkBioBank
Clicked point ID: sub-5445040, Dataset: UkBioBank
Clicked point ID: sub-2864945, Dataset: UkBioBank
Clicked point ID: sub-4934833, Dataset: UkBioBank
Clicked point ID: sub-2046717, Dataset: UkBioBank
Clicked point ID: sub-4869482, Dataset: UkBioBank
Clicked point ID: sub-1067773, Dataset: UkBioBank
Clicked point ID: sub-4216589, Dataset: UkBioBank
Clicked point ID: sub-2203842, Dataset: UkBioBank


In [127]:
side = "R"
region = "CINGULATE"

bucket_files = []

for subject_id, dataset in zip(subject_id_list,dataset_name_list):
    if dataset.lower() in ['ukb', 'ukbiobank']:
        dataset = 'UkBioBank'
        path = f'/neurospin/dico/data/deep_folding/current/datasets/{dataset}/crops/2mm/{region}/mask/{side}buckets'

    if dataset.lower() in ['hcp']:
        path = f'/neurospin/dico/data/deep_folding/current/datasets/{dataset.lower()}/crops/2mm/{region}/mask/{side}buckets'

    filename = f'{path}/{subject_id}_cropped_skeleton.bck'#.minf'

    if os.path. isfile(filename):
        bucket_files.append(filename)
    else:
        print(f"{filename} is not a correct path, or the .bck doesn't exist")
bucket_files

['/neurospin/dico/data/deep_folding/current/datasets/UkBioBank/crops/2mm/CINGULATE/mask/Rbuckets/sub-1160330_cropped_skeleton.bck',
 '/neurospin/dico/data/deep_folding/current/datasets/UkBioBank/crops/2mm/CINGULATE/mask/Rbuckets/sub-2938526_cropped_skeleton.bck',
 '/neurospin/dico/data/deep_folding/current/datasets/UkBioBank/crops/2mm/CINGULATE/mask/Rbuckets/sub-4595658_cropped_skeleton.bck',
 '/neurospin/dico/data/deep_folding/current/datasets/UkBioBank/crops/2mm/CINGULATE/mask/Rbuckets/sub-5445040_cropped_skeleton.bck',
 '/neurospin/dico/data/deep_folding/current/datasets/UkBioBank/crops/2mm/CINGULATE/mask/Rbuckets/sub-2864945_cropped_skeleton.bck',
 '/neurospin/dico/data/deep_folding/current/datasets/UkBioBank/crops/2mm/CINGULATE/mask/Rbuckets/sub-4934833_cropped_skeleton.bck',
 '/neurospin/dico/data/deep_folding/current/datasets/UkBioBank/crops/2mm/CINGULATE/mask/Rbuckets/sub-2046717_cropped_skeleton.bck',
 '/neurospin/dico/data/deep_folding/current/datasets/UkBioBank/crops/2mm/CIN

In [10]:
import anatomist.api as ana
from soma.qt_gui.qtThread import QtThreadCall
from soma.qt_gui.qt_backend import Qt

a = ana.Anatomist()

create qapp
global modules: /casa/host/build/share/anatomist-5.2/python_plugins
home   modules: /casa/home/.anatomist/python_plugins
done
Starting Anatomist.....
config file : /casa/home/.anatomist/config/settings.cfg
PyAnatomist Module present
PythonLauncher::runModules()
loading module simple_controls
loading module save_resampled


existing QApplication: 0
QStandardPaths: XDG_RUNTIME_DIR not set, defaulting to '/tmp/runtime-ad279118'


loading module selection
loading module bsa_proba
loading module modelGraphs
loading module profilewindow
loading module ana_image_math
loading module paletteViewer
loading module foldsplit
loading module anacontrolmenu
loading module gradientpalette
loading module palettecontrols
loading module meshsplit
loading module volumepalettes
loading module gltf_io
loading module infowindow
loading module histogram
loading module statsplotwindow
loading module valuesplotwindow
all python modules loaded
Anatomist started.


In [128]:
block = a.createWindowsBlock(4) # 4 columns
d = {}

for i, file in enumerate(bucket_files):
    d[f'bck_{i}'] = a.loadObject(file)
    d[f'w_{i}'] = a.createWindow('3D', block=block)#geometry=[100+400*(i%3), 100+440*(i//3), 400, 400])
    d[f'w_{i}'].addObjects(d[f'bck_{i}'])

In [12]:
from soma import aims

In [129]:
dic_vol = {}
dim = 0
rep = 0
while dim == 0 and rep < len(subject_id_list):
    mm_skeleton_path = f"/neurospin/dico/data/deep_folding/current/datasets/{dataset_name_list[rep]}/crops/2mm/{region}/mask/{side}crops"
    if os.path. isfile(f'{mm_skeleton_path}/{subject_id_list[rep]}_cropped_skeleton.nii.gz'):
        dim = aims.read(f'{mm_skeleton_path}/{subject_id_list[rep]}_cropped_skeleton.nii.gz').np.shape
        sum_vol = np.zeros(shape=dim)
    else: 
        print(f'FileNotFound {mm_skeleton_path}/{subject_id_list[rep]}_cropped_skeleton.nii.gz')
        #raise FileNotFoundError(f'{mm_skeleton_path}/{subject_id_list[0]}_cropped_skeleton.nii.gz')
    rep += 1

for subject_id, dataset in zip(subject_id_list,dataset_name_list):
    if dataset.lower() in ['ukb', 'ukbiobank']:
        dataset = 'UkBioBank'
    elif dataset.lower() == 'hcp':
        dataset = 'hcp'
        
    mm_skeleton_path = f"/neurospin/dico/data/deep_folding/current/datasets/{dataset}/crops/2mm/{region}/mask/{side}crops"

    if os.path. isfile(f'{mm_skeleton_path}/{subject_id}_cropped_skeleton.nii.gz'):
        vol = aims.read(f'{mm_skeleton_path}/{subject_id}_cropped_skeleton.nii.gz')
        # compare the dim with the first file dim

        if vol.np.shape != dim:
            raise ValueError(f"{subject_id_list[0]} and {subject_id} must have the same dim")

            
        # to have a binary 3D structure
        dic_vol[subject_id] = (vol.np > 0).astype(int) 
        sum_vol += (vol.np > 0).astype(int) 
    else: 
        print(f'FileNotFound {mm_skeleton_path}/{subject_id}_cropped_skeleton.nii.gz')
        #raise FileNotFoundError(f'{mm_skeleton_path}/{subject_id}_cropped_skeleton.nii.gz')

sum_vol = sum_vol/len(subject_id_list)
#print(dic_vol[subject_id_list[0]].shape)
#print(np.count_nonzero(dic_vol[subject_id_list[0]]))

In [130]:
# Create axis
axes = list(dim[:3])
dim1 = list(dim[:3])[0]
dim2 = list(dim[:3])[1]
dim3 = list(dim[:3])[2]

# Create Data
data = sum_vol.reshape(list(dim[:3]))

X, Y, Z = np.mgrid[0:dim1:1, 0:dim2:1, 0:dim3:1]
values = np.flip(data, axis=[1,2])

fig = go.Figure(data=go.Volume(
    x=X.flatten(),
    y=Y.flatten(),
    z=Z.flatten(),
    value=values.flatten(),
    isomin=0.15,
    isomax=1,
    opacity=0.1, # needs to be small to see through all surfaces
    surface_count=17, # needs to be a large number for good volume rendering
    ))
fig.show()

Position : 10.8721, 24.3388, 42.3501, 0


In [ ]:
"""
data_filenames_str = ' '.join(data_filenames_list)

#os.system(f'anatomist --input {data_filenames_str}')

# Command to run the Python script inside the container
command = f'bv bash -c "python visu_anatomist.py {data_filenames_str}"'

# Execute the command
os.system(command)
"""

In [31]:
# See for more information
# https://plotly.com/python/line-and-scatter/
# https://plotly.com/python/setting-graph-size/
# app.run_server(debug=True)